## Homework5-0314
   **截止日期：**<font color=ff0000>**3.17(周四)晚** </font>
   <br><br>**注：**本次作业中完成对应题目的选做题可以不用完成原题

### 1 正则表达式(2分)：
具体格式要求如下：
<br>账号：由英文字母组成，不限大小写。
<br>密码：由英文字母与数字组成且以英文字母开头，最少八位。
<br>电话：11位数字，且以1开头。
<br>生日：年-月-日(YYYY-MM-DD)。

In [9]:
import re
account="[a-zA-Z]+"
password="[a-zA-Z][a-zA-Z\d]{7,}"
telephone="1\d{10}"
birthday="\d{4}\-\d{2}|\-\d{2}"
#TODO

assert re.match(account,"PyThoN")
assert re.match(password,"Python2022")
assert re.match(telephone,"12345678910")
assert re.match(birthday,"2022-03-14")

### 1.5(选做)邮箱的正则表达式匹配（2+0.5分）
邮箱中间须有@，@之前由英文字母、数字、下划线(_)、短横(-)组成，之后由英文字母、数字、下划线(_)、短横(-)、英文句号(.)组成，且必须有至少一个英文句号(.)。（0.5分）

In [10]:
email="[a-zA-Z\d\_\-]+\@[a-zA-Z\d\_\-]+\.[a-zA-Z\d\_\-\.]+"
assert re.match(email,"2-102@pku.edu.cn")

### 2 yield初步（2分)：
使用yield实现有以下隐含功能的迭代器：默认情况返回给定字符串中的下一个单词；如果send一个字符串，则将其改为全小写并返回。

In [1]:
s="Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language and first released it in 1991 as Python 0.9.0."
#TODO
def gen_cal():
    x = 0
    l = s.split()
    length = len(l)
    msg = None
    while x < length:
        if msg == None:
            msg = yield l[x]
            x += 1
        else:
            msg = yield msg.lower()
gc = gen_cal()
print(next(gc))
print(next(gc))
print(gc.send("ASASD"))
print(next(gc))

Guido
van
asasd
Rossum


### 3 async/await初步（2分）
采用异步非并发与并发的方式完成代码，并比较完成时间。

In [9]:
import asyncio,time
async def working(time):
    await asyncio.sleep(time)
    
async def worker_1():
    task_list=[5,1,3]
    for task in task_list:
        await working(task)
    
async def worker_2():
    task_list=[3,1,5]
    for task in task_list:
        await working(task)

async def main():
    print(f"started at{time.strftime('%X')}")
    #TODO
    await worker_1()
    await worker_2()
    print(f"finished at{time.strftime('%X')}")
    task1 = asyncio.create_task(worker_1())
    task2 = asyncio.create_task(worker_2())
    await task1
    await task2
    print(f"finished at{time.strftime('%X')}")

await main()


started at13:44:33
finished at13:44:51
finished at13:45:00


### 3.5(选做) async/await初步(2+0.5分)
(选做)(+0.5)如果worker_1只能完成耗时为1或3的任务，worker_2只能完成耗时为5的任务，请完成以下功能：worker发现自己无法完成当前任务，就把该任务交给另一个worker完成（只需修改task_list即可），代码可随意修改。

In [1]:
import asyncio,time
async def working(time):
    await asyncio.sleep(time)
    
async def worker_1(task_list=[5,1,3]):

    for task in task_list:
        if task == 1 or task == 3:
            await working(task)
        else:
            await worker_2([task])
    
async def worker_2(task_list = [3,1,5]):
    for task in task_list:
        if task == 5:
            await working(task)
        else:
            await worker_1([task])

async def main():
    print(f"started at{time.strftime('%X')}")
    #TODO
    task1 = asyncio.create_task(worker_1())
    task2 = asyncio.create_task(worker_2())
    await task1
    await task2
    print(f"finished at{time.strftime('%X')}")

await main()



started at11:40:57
finished at11:41:06


### 4 协程实现轮转调度（2分）
在课件上的轮转调度框架的基础上，完成题目：
<br>worker1与worker2分别有三种不同等级（1,2,3）的任务，task中储存了这几种任务对应的数目（例如worker1有3个等级为1的任务），worker1与worker2一次只可完成一个任务。
<br>请实现：Handle可指定一种等级(例如等级2)的任务，worker1与worker2只需完成该等级的所有任务。
    



In [2]:
waiting_list=[]

def worker1():
    task={1:3,2:1,3:3}
    rank=yield
    while task[rank]>0:
        task[rank]=task[rank]-1
        print(f'worker1:{task[rank]} remains.')
        yield
    print('worker1 finished!')
    
def worker2():
    task={1:2,2:3,3:3}
    rank=yield
    while task[rank]>0:
        task[rank]=task[rank]-1
        print(f'worker2:{task[rank]} remains.')
        yield
    print('worker2 finished!')
    
    
class Handle(object):
    
    def __init__(self,gen):
        self.gen=gen
        #next(self.gen)

    def call(self):
        #TODO
        try:
            self.gen.send(2)
        except TypeError:
            next(self.gen)
        #waiting_list.append(self)
    
def RoundRobin(*worker):
    waiting_list.extend(Handle(c) for c in worker)
    #TODO
    while waiting_list:
        for p in waiting_list:
            try:
                p.call()
            except StopIteration:
                waiting_list.remove(p)
                
    print('All finished!')
    
w1=worker1()
w2=worker2()

RoundRobin(w1,w2)

worker1:0 remains.
worker2:2 remains.
worker1 finished!
worker2:1 remains.
worker2:0 remains.
worker2 finished!
All finished!


### 4.5(选做) 协程实现轮转调度（2+0.5分）：
worker1与worker2按照1-2-3的等级依次完成任务，如果当前等级任务数目为0，则输出“Worker1/2 takes a break.”(认为其完成了该等级任务，不影响循环进行)，如果worker1或worker2已完成所有任务，则由剩下的一个worker独自按照1-2-3的顺序完成任务。例如：worker1完成1级任务->worker2完成2级任务->worker1完成3级任务->worker2完成1级任务……（代码可随意修改！）

In [16]:
waiting_list=[]

def worker1():
    task={1:3,2:1,3:3}
    s = sum(task.values())
    while s >0:
        rank = yield
        if task[rank] > 0:
            task[rank] -= 1
            s -= 1
            print("worker1 finish {} task",rank)
        else:
            print("worker1 takes a break")
    print('worker1 finished!')
    
def worker2():
    task={1:2,2:3,3:3}
    s = sum(task.values())
    while s >0:
        rank = yield
        if task[rank] > 0:
            task[rank] -= 1
            s -= 1
            print("worker2 finish {} task",rank)
        else:
            print("worker2 takes a break")
    print('worker2 finished!')
    
    
class Handle(object):
    
    def __init__(self,gen):
        self.gen=gen
        #next(self.gen)

    def call(self,typ):
        #TODO
        #print(typ)
        try:
            self.gen.send(typ)
            return 1
        except TypeError:
            next(self.gen)
            return 0
        #waiting_list.append(self)
    
def RoundRobin(*worker):
    waiting_list.extend(Handle(c) for c in worker)
    #TODO
    typ = 1;
    while waiting_list:
        for p in waiting_list:
            try:
                t = p.call(typ)
                typ = typ % 3 + t;
            except StopIteration:
                waiting_list.remove(p)
                
    print('All finished!')
    
w1=worker1()
w2=worker2()

RoundRobin(w1,w2)

worker1 finish {} task 1
worker2 finish {} task 2
worker1 finish {} task 3
worker2 finish {} task 1
worker1 finish {} task 2
worker2 finish {} task 3
worker1 finish {} task 1
worker2 finish {} task 2
worker1 finish {} task 3
worker2 finish {} task 1
worker1 takes a break
worker2 finish {} task 3
worker1 finish {} task 1
worker2 finish {} task 2
worker1 finish {} task 3
worker1 finished!
worker2 finish {} task 3
worker2 finished!
All finished!


### 5 生产者/消费者（2分）
采用生产者消费者模型完成给定数据集的检索。这里生产者将检索字符串放入队列中，消费者读取并查找包含字符串的文章，返回给生产者。
<br>需查找：包含（1）全国XX委员会（2）中国XX协会（3）中美建交

In [6]:
news=[]
with open('news.txt','r',encoding='utf-8') as f:
    news=f.readlines()

In [2]:
news=[]
with open('news.txt','r',encoding='utf-8') as f:
    news=f.readlines()
import nest_asyncio
import asyncio,time
import re
a = '.{0,}全国[\u4e00-\u9fa5]{0,}委员会.{0,}'
b = '.{0,}中国[\u4e00-\u9fa5]{0,}协会.{0,}'
c = '.{0,}中美建交.{0,}'
nest_asyncio.apply() 
A = []
B = []
C = []
async def consumer(q):
    print('consumer starts.')
    #TODO
    while True:
        x = await q.get()
        #print(x)
        if x is None:
            q.task_done()
            break
        else:
            #print(x)
            if re.match(a,x):
                A.append(x)
            if re.match(b,x):
                B.append(x)
            if re.match(c,x):
                C.append(x)
              #  pass
            q.task_done()
    print("consumer ends.")
    
async def producer(q):
    print('producer starts.')
    #TODO
    for x in news:
        await q.put(x)
    await q.put(None)
    await q.join()
    print(A,B,C)
    print("producer ends.")
    
q=asyncio.Queue(maxsize=10)
t0=time.time()
loop=asyncio.get_event_loop()
tasks=[producer(q),consumer(q)]
loop.run_until_complete(asyncio.wait(tasks))

print(time.time()-t0,"s")

producer starts.
consumer starts.
consumer ends.
['（一九七九年一月一日）第五届全国人民代表大会常务委员会于一九七八年十二月二十六日举行第五次会议讨论通过了《中华人民共和国全国人民代表大会常务委员会告台湾同胞书》＊＊＊亲爱的台湾同胞：今天是一九七九年元旦。我们代表祖国大陆的各族人民，向诸位同胞致以亲切的问候和衷心的祝贺。昔人有言：“每逢佳节倍思亲”。在这欢度新年的时刻，我们更加想念自己的亲骨肉——台湾的父老兄弟姐妹。我们知道，你们也无限怀念祖国和大陆上的亲人。这种绵延了多少岁月的相互思念之情与日俱增。自从一九四九年台湾同祖国不幸分离以来，我们之间音讯不通，来往断绝，祖国不能统一，亲人无从团聚，民族、国家和人民都受到了巨大的损失。所有中国同胞以及全球华裔，无不盼望早日结束这种令人痛心的局面。我们中华民族是伟大的民族，占世界人口近四分之一，享有悠久的历史和优秀的文化，对世界文明和人类发展的卓越贡献，举世共认。台湾自古就是中国不可分割的一部分。中华民族是具有强大的生命力和凝聚力的。尽管历史上有过多少次外族入侵和内部纷争，都不曾使我们的民族陷于长久分裂。近三十年台湾同祖国的分离，是人为的，是违反我们民族的利益和愿望的，决不能再这样下去了。每一个中国人，不论是生活在台湾的还是生活在大陆上的，都对中华民族的生存、发展和繁荣负有不容推诿的责任。统一祖国这样一个关系全民族前途的重大任务，现在摆在我们大家的面前，谁也不能回避，谁也不应回避。如果我们还不尽快结束目前这种分裂局面，早日实现祖国的统一，我们何以告慰于列祖列宗？何以自解于子孙后代？人同此心，心同此理，凡属黄帝子孙，谁愿成为民族的千古罪人？近三十年来，中国在世界上的地位已发生根本变化。我国国际地位越来越高，国际作用越来越重要。各国人民和政府为了反对霸权主义、维护亚洲和世界的和平稳定，几乎莫不对我们寄予极大期望。每一个中国人都为祖国的日见强盛而感到自豪。我们如果尽快结束目前的分裂局面，把力量合到一起，则所能贡献于人类前途者，自更不可限量。早日实现祖国统一，不仅是全中国人民包括台湾同胞的共同心愿，也是全世界一切爱好和平的人民和国家的共同希望。今天，实现中国的统一，是人心所向，大势所趋。世界上普遍承认只有一个中国，承认中华人民共和国政府是中国唯一合法的政府。最近中日和平友好条约的签定

### 5.5(选做) 生产者/消费者（2+0.5分）
完成本地C/S架构的生产者消费者模型,并完成以上任务。

In [ ]:
#TODO